# **GANs: Redes Generativas Adversarias**

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook
import tensorflow
%matplotlib inline

### **Obtener la data**

In [ ]:
from tensorflow.keras.datasets import mnist

(trainX, trainY), (testX, testY) = mnist.load_data()

print('Data de entrenamiento: X=%s, y=%s' % (trainX.shape, trainY.shape))
print('Data de test: X=%s, y=%s' % (testX.shape, testY.shape))

In [ ]:
plt.figure(figsize=(7, 7))
for j in range(25):
  i = np.random.randint(0, trainX.shape[0])
  plt.subplot(5,5, j+1 )
  plt.imshow(trainX[i], cmap='gray_r')
  plt.axis('off')
plt.show()

### **Normalizar la data**

In [ ]:
trainX = [image/255.0 for image in trainX]
testX = [image/255.0 for image in testX]

trainX = np.reshape(trainX, (60000, 28, 28, 1))
testX = np.reshape(testX, (10000, 28, 28, 1))

print (trainX.shape, testX.shape, trainY.shape, testY.shape)

### **Generador**

In [ ]:
from keras.layers import Dense, BatchNormalization, Reshape, Activation, Flatten, Input, Dropout
from keras.models import Model

random_input = Input(shape = 50)

x = Dense(1200, activation='relu')(random_input)
x = BatchNormalization(momentum=0.8)(x)
x = Dense(1000, activation='relu')(x)
x = BatchNormalization(momentum=0.8)(x)
x = Dense(28*28)(x)
x = Reshape((28, 28, 1))(x)

generated_image = Activation('sigmoid')(x)

generator_network = Model(inputs=random_input, outputs=generated_image)
generator_network.summary()

### **Discriminador**

In [ ]:
image_input = Input(shape=(28, 28, 1))

x = Flatten()(image_input)
x = Dense(256, activation='relu')(x)
x = Dropout(0.3)(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(1)(x)

real_vs_fake_output = Activation('sigmoid')(x)

discriminator_network = Model(inputs=image_input, outputs=real_vs_fake_output)
discriminator_network.summary()

In [ ]:
from tensorflow.keras.optimizers.legacy import Adam
adam_optimizer = Adam(learning_rate=0.00005, beta_1=0.5)
discriminator_network.compile(loss='binary_crossentropy', optimizer=adam_optimizer, metrics=['accuracy'])

### Modelo combinando ambas redes

In [ ]:
discriminator_network.trainable=False

g_output = generator_network(random_input)
d_output = discriminator_network(g_output)

gan_model = tensorflow.keras.models.Model(random_input, d_output)
gan_model.summary()

In [ ]:
gan_model.compile(loss='binary_crossentropy', optimizer=adam_optimizer)

### Objetos para acceder a la data para el entrenamiento

In [ ]:
indices = [i for i in range(len(trainX))]

def get_random_noise(batch_size, noise_size):
    random_values = np.random.randn(batch_size*noise_size)
    random_noise_batch = np.reshape(random_values, (batch_size, noise_size))
    return random_noise_batch

def get_fake_samples(generator_network, batch_size, noise_size):
    random_noise_batch = get_random_noise(batch_size, noise_size)
    fake_samples = generator_network.predict_on_batch(random_noise_batch)
    return fake_samples

def get_real_samples(batch_size):
    random_indices = np.random.choice(indices, size=batch_size)
    real_images = trainX[np.array(random_indices),:]
    return real_images

def show_generator_results(generator_network):
    for k in range(9):
        plt.figure(figsize=(7, 7))
        fake_samples = get_fake_samples(generator_network, 9, noise_size)
        for j in range(9):
            plt.subplot(990 + 1 + j)
            plt.imshow(fake_samples[j,:,:,-1], cmap='gray_r')
            plt.axis('off')
        plt.show()
    return

### Entrenar el GAN

In [ ]:
epochs = 20
batch_size = 200
steps =  500
noise_size = 50
print_every = 10

losses_d = []
losses_g = []

for i in range(0, epochs):
    if (i%print_every == 0):
        show_generator_results(generator_network)
    for j in range(steps):
        fake_samples = get_fake_samples(generator_network, batch_size//2, noise_size)
        real_samples = get_real_samples(batch_size=batch_size//2)

        fake_y = np.zeros((batch_size//2, 1))
        real_y = np.ones((batch_size//2, 1))

        input_batch = np.vstack((fake_samples, real_samples))
        output_labels = np.vstack((fake_y, real_y))

        discriminator_network.trainable=True
        loss_d = discriminator_network.train_on_batch(input_batch, output_labels)

        gan_input = get_random_noise(batch_size, noise_size)
        gan_output = np.ones((batch_size))

        discriminator_network.trainable=False
        loss_g = gan_model.train_on_batch(gan_input, gan_output)

        losses_d.append(loss_d[0])
        losses_g.append(loss_g)

        if j%50 == 0:
            print ("Epoch:%.0f, Step:%.0f, D-Loss:%.3f, D-Acc:%.3f, G-Loss:%.3f"%(i,j,loss_d[0],loss_d[1]*100,loss_g))

# Resultados

In [ ]:
for i in range(5):
    show_generator_results(generator_network)

### Curvas de Aprendizaje

In [ ]:
steps = [i for i in range(len(losses_d))]
plt.figure(figsize=(10, 6))
plt.plot(losses_d[:5000])
plt.plot(losses_g[:5000])
plt.xlabel('Steps')
plt.ylabel('Loss Value')
plt.title("GAN: Función de Pérdida")
plt.legend(['D-Loss', 'G-Loss'])
plt.show()